In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [ ]:
print(dataset.columns)

In [ ]:
data_url = '/kaggle/input/world-income-inequality-database'
dataset = pd.read_csv('/kaggle/input/world-income-inequality-database/WIID_06MAY2020.csv')

# Preprocess data
dataset = dataset.dropna(subset=['gini_reported', 'gdp_ppp_pc_usd2011'])
data = dataset[['gini_reported', 'gdp_ppp_pc_usd2011']].values
data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))  # Normalize



In [ ]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(2, activation='tanh'))
    return model

def build_discriminator():
    model = Sequential()
    model.add(Dense(128, input_dim=2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [ ]:
# Compile and train GAN

# Hyperparameters
latent_dim = 100
batch_size = 64
epochs = 200

# Build and compile discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile combined model
generator = build_generator(latent_dim)
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

import warnings
warnings.filterwarnings("ignore")

# Train the GAN
real_labels = np.ones((batch_size, 1))
fake_labels = np.zeros((batch_size, 1))

for epoch in range(epochs):
    # Train the discriminator
    idx = np.random.randint(0, data.shape[0], batch_size)
    real_data = data[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_data = generator.predict(noise)

    real_loss = discriminator.train_on_batch(real_data, real_labels)
    fake_loss = discriminator.train_on_batch(fake_data, fake_labels)
    d_loss = 0.5 * np.add(real_loss, fake_loss)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, real_labels)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]:.4f}, acc.: {100 * d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")


In [ ]:
# Generate synthetic data
num_samples = 1000
noise = np.random.normal(0, 1, (num_samples, latent_dim))
synthetic_data = generator.predict(noise)

# Rescale synthetic data to the range [0, 1]
synthetic_data = (synthetic_data + 1) / 2

# Visualize synthetic and real data
plt.scatter(data[:, 0], data[:, 1], label='Real Data', alpha=0.5)
plt.scatter(synthetic_data[:, 0], synthetic_data[:, 1], label='Synthetic Data', alpha=0.5)
plt.xlabel('Gini Index')
plt.ylabel('Income per Capita')
plt.legend()
plt.show()

# Unit test
def test_synthetic_data(synthetic_data, num_samples, data_shape):
    assert synthetic_data.shape == (num_samples, data_shape[1]), f"Expected shape {(num_samples, data_shape[1])}, but got {synthetic_data.shape}"
    assert np.all((synthetic_data >= 0) & (synthetic_data <= 1)), "Synthetic data should be within the range [0, 1]"

# Run unit test
test_synthetic_data(synthetic_data, num_samples, data.shape)